# Assignment 7

###### Created by Qihang Ma -- 2023.04.26

In [72]:
import warnings
warnings.filterwarnings("ignore")
from risk_mgmt import VaR, calculation, simulation
import pandas as pd
import numpy as np
import datetime as dt
from scipy.optimize import fsolve, minimize
import statsmodels.api as sm

## Problem 1 - Calculate the Ex-Post Return Attribution for each Stock.

Part 1 – Calculate the maximum SR portfolio using the method from last week’s homework for the following stocks: AAPL, MSFT, BRK-B, CSCO, and JNJ.

Use the returns from the end of the history (1-14) until the end of February. 

Calculate the Ex-Post Return Attribution for each Stock.

Part 2 – Using the same data as part 1, add the risk attribution for each stock:

In [73]:
ff = pd.read_csv('F-F_Research_Data_Factors_daily.csv', parse_dates=['Date']).set_index('Date')
mom = pd.read_csv('F-F_Momentum_Factor_daily.csv', parse_dates=['Date']).set_index('Date').rename(columns={'Mom   ':  "Mom"})

factor = (ff.join(mom, how='right') / 100)

In [74]:
all_returns = pd.read_csv('DailyReturn.csv', parse_dates=['Date']).set_index('Date')


In [75]:
stocks = ['AAPL',  'MSFT' , 'BRK-B', 'JNJ', 'CSCO']
factors = ['Mkt-RF', 'SMB', 'HML', 'Mom']
dataset = pd.merge(all_returns[stocks], factor, left_index=True, right_index=True)

subset = dataset.dropna()

### Find Alpha and Beta

In [76]:
X = subset[factors]
X = sm.add_constant(X)

y = subset[stocks].sub(subset['RF'], axis=0)

betas = pd.DataFrame(index=stocks, columns=factors)
alphas = pd.DataFrame(index=stocks, columns=['Alpha'])


for stock in stocks:
    model = sm.OLS(y[stock], X).fit()
    betas.loc[stock] = model.params[factors]
    alphas.loc[stock] = model.params['const']

### Calculate the Expected Return

In [77]:
max_date = max(all_returns.index)
min_date = max_date - dt.timedelta(days= 10*365)
expected_Factor_return = pd.DataFrame(np.mean(factor[factors].loc[min_date:max_date]))

expected_annual_return = pd.DataFrame((np.dot(betas.values, expected_Factor_return.values)).T, columns=[stocks]).applymap(lambda x: np.log(1+x)*255)
expected_annual_return

,AAPL,MSFT,BRK-B,JNJ,CSCO
0,0.141728,0.170172,0.110254,0.071027,0.136489


In [78]:
covariance_matrix = dataset[stocks].applymap(lambda x: np.log(1+x)).cov() * 255
covariance_matrix

,AAPL,MSFT,BRK-B,JNJ,CSCO
AAPL,0.065441,0.039993,0.000136,-0.003238,0.011880
MSFT,0.039993,0.065237,-0.001568,-0.001555,0.022863
BRK-B,0.000136,-0.001568,0.022978,0.009128,0.007915
JNJ,-0.003238,-0.001555,0.009128,0.022242,0.005765
CSCO,0.011880,0.022863,0.007915,0.005765,0.054527


### Find Optimal Weights

In [79]:
def optimize_weights(stocks, stockMeans, covar, rf):

    # Define Sharpe ratio function
    def sr(w):
        m = np.dot(w, stockMeans) - rf
        s = np.sqrt(np.dot(w.T, np.dot(covar, w)))
        return m / s

    n = len(stocks)

    # Define optimization problem
    bounds = [(0, None)] * n
    cons = {"type": "eq", "fun": lambda w: np.sum(w) - 1}
    result = minimize(lambda w: -sr(w), np.ones(n) / n, method="SLSQP", bounds=bounds, constraints=cons)

    # Extract optimal weights and other information
    w = result.x
    w = w / np.sum(w)
    OptWeights = pd.DataFrame({"Stock": stocks, "Weight": w, "cEr": stockMeans * w})

    return OptWeights


In [80]:
OptWeight = optimize_weights(stocks, expected_annual_return.values[0], covariance_matrix, .0025)
OptWeight

,Stock,Weight,cEr
0,AAPL,0.100792,0.014285
1,MSFT,0.209487,0.035649
2,BRK-B,0.438283,0.048322
3,JNJ,0.170222,0.012090
4,CSCO,0.081217,0.011085


### Calculate Ex-post Return & Risk attribution

In [81]:
update_price = pd.read_csv("updated_prices.csv", parse_dates=['Date'])
upReturns = calculation.return_calculate(update_price)

In [82]:
def expost_attribution(w, upReturns):
    n = upReturns.shape[0]
    pReturn = np.empty(n)
    weights = np.empty((n, len(w)))
    lastW = np.copy(w)
    matReturns = upReturns[stocks].values

    for i in range(n):
        # Save Current Weights in Matrix
        weights[i,:] = lastW

        # Update Weights by return
        lastW = lastW * (1.0 + matReturns[i,:])

        # Portfolio return is the sum of the updated weights
        pR = np.sum(lastW)
        # Normalize the wieghts back so sum = 1
        lastW = lastW / pR
        # Store the return
        pReturn[i] = pR - 1

    # Set the portfolio return in the Update Return DataFrame
    upReturns['Portfolio'] = pReturn

    # Calculate the total return
    totalRet = np.exp(np.sum(np.log(pReturn + 1))) - 1
    # Calculate the Carino K
    k = np.log(totalRet + 1) / totalRet

    # Carino k_t is the ratio scaled by 1/K 
    carinoK = np.log(1.0 + pReturn) / pReturn / k
    # Calculate the return attribution
    attrib = pd.DataFrame(matReturns * (weights * carinoK[:, np.newaxis]), columns=stocks)

    # Set up a Dataframe for output.
    Attribution = pd.DataFrame({'Value': ["TotalReturn", "Return Attribution"]})


    for s in stocks + ['Portfolio']:
        # Total Stock return over the period
        tr = np.exp(np.sum(np.log(upReturns[s] + 1))) - 1
        # Attribution Return (total portfolio return if we are updating the portfolio column)
        atr =  attrib[s].sum() if s != 'Portfolio' else tr
        # Set the values
        Attribution[s] = [tr, atr]

        # Y is our stock returns scaled by their weight at each time
        Y =  matReturns * weights
        # Set up X with the Portfolio Return
        X = np.column_stack((np.ones((pReturn.shape[0], 1)), pReturn))
        # Calculate the Beta and discard the intercept
        B = (np.linalg.inv(X.T @ X) @ X.T @ Y)[1,:]
        # Component SD is Beta times the standard Deviation of the portfolio
        cSD = B * np.std(pReturn)

        Expost_Attribution = pd.concat([Attribution,    
            pd.DataFrame({"Value": ["Vol Attribution"], 
                        **{stocks[i]: [cSD[i]] for i in range(len(stocks))},
                        "Portfolio": [np.std(pReturn)]})
        ], ignore_index=True)

    return Expost_Attribution


In [83]:
expost_attribution(OptWeight['Weight'].values, upReturns[stocks])

,Value,AAPL,MSFT,BRK-B,JNJ,CSCO,Portfolio
0,TotalReturn,-0.044720,-0.034791,-0.008268,-0.013189,-0.091102,-0.025063
1,Return Attribution,-0.004590,-0.007082,-0.003693,-0.002152,-0.007547,-0.025063
2,Vol Attribution,0.001565,0.003159,0.004690,0.001409,0.000895,0.011718


## Problem2 - Factor Attribution 

Using the same data as Problem 1, attribute realized risk and return to the Fama French 3+Momentum model. Report the residual total as Portfolio Alpha.

In [84]:
OptWeight

,Stock,Weight,cEr
0,AAPL,0.100792,0.014285
1,MSFT,0.209487,0.035649
2,BRK-B,0.438283,0.048322
3,JNJ,0.170222,0.012090
4,CSCO,0.081217,0.011085


In [85]:
stocks = ['AAPL',  'MSFT' , 'BRK-B', 'JNJ', 'CSCO']
factors = ['Mkt-RF', 'SMB', 'HML', 'Mom']

In [86]:
update_price = pd.read_csv("updated_prices.csv", parse_dates=['Date'])
upReturns = calculation.return_calculate(update_price)

In [87]:
ff = pd.read_csv('updated_F-F_Research_Data_Factors_daily.CSV', parse_dates=['Date']).set_index('Date')
mom = pd.read_csv('updated_F-F_Momentum_Factor_daily.csv', parse_dates=['Date']).set_index('Date').rename(columns={'Mom   ':  "Mom"})

updated_factor = (ff.join(mom, how='right') / 100)
updated_factor = pd.concat([factor,updated_factor])

updated_ff_data = pd.merge(updated_factor, upReturns, left_index=True, right_on='Date')[factors]

In [88]:
def expost_factor(w, upReturns, upFfData, Betas):
    stocks = upReturns.columns
    factors = list(upFfData.columns)
    
    n = upReturns.shape[0]
    m = len(stocks)
    
    pReturn = np.empty(n)
    residReturn = np.empty(n)
    weights = np.empty((n, len(w)))
    factorWeights = np.empty((n, len(factors)))
    lastW = w.copy()
    matReturns = upReturns[stocks].to_numpy()
    ffReturns = upFfData[factors].to_numpy()

    for i in range(n):
        # Save Current Weights in Matrix
        weights[i,:] = lastW

        #Factor Weight
        factorWeights[i,:] = Betas.T @ lastW

        # Update Weights by return
        lastW = lastW * (1.0 + matReturns[i,:])

        # Portfolio return is the sum of the updated weights
        pR = np.sum(lastW)
        # Normalize the weights back so sum = 1
        lastW = lastW / pR
        # Store the return
        pReturn[i] = pR - 1

        # Residual
        residReturn[i] = (pR-1) - factorWeights[i,:] @ ffReturns[i,:]

    # Set the portfolio return in the Update Return DataFrame
    upFfData["Alpha"] = residReturn
    upFfData["Portfolio"] = pReturn

    # Calculate the total return
    totalRet = np.exp(np.sum(np.log(pReturn + 1))) - 1
    # Calculate the Carino K
    k = np.log(totalRet + 1) / totalRet

    # Carino k_t is the ratio scaled by 1/K 
    carinoK = np.log(1.0 + pReturn) / pReturn / k
    # Calculate the return attribution
    attrib = pd.DataFrame(ffReturns * (factorWeights * carinoK[:, np.newaxis]), columns=factors)
    attrib["Alpha"] = residReturn * carinoK

    # Set up a DataFrame for output.
    Attribution = pd.DataFrame({"Value": ["TotalReturn", "Return Attribution"]})

    newFactors = factors + ['Alpha']
    # Loop over the factors
    for s in newFactors + ["Portfolio"]:
        # Total Stock return over the period
        tr = np.exp(np.sum(np.log(upFfData[s] + 1))) - 1
        # Attribution Return (total portfolio return if we are updating the portfolio column)
        atr = sum(attrib[s]) if s != "Portfolio" else tr
        # Set the values
        Attribution[s] = [tr, atr]

    # Realized Volatility Attribution

    # Y is our stock returns scaled by their weight at each time
    Y = np.hstack((ffReturns * factorWeights, residReturn[:,np.newaxis]))
    # Set up X with the Portfolio Return
    X = np.hstack((np.ones((n,1)), pReturn[:,np.newaxis]))
    # Calculate the Beta and discard the intercept
    B = np.linalg.inv(X.T @ X) @ X.T @ Y
    B = B[1,:]
    # Component SD is Beta times the standard Deviation of the portfolio
    cSD = B * np.std(pReturn)

    # Check that the sum of component SD is equal to the portfolio SD
    assert np.isclose(np.sum(cSD), np.std(pReturn))

    # Add the Vol attribution to the output
    Expost_Attribution = pd.concat([Attribution, 
        pd.DataFrame({"Value": "Vol Attribution", **{newFactors[i]:cSD[i] for i in range(len(newFactors))}, "Portfolio":np.std(pReturn)}, index=[0])
    ])

    return Expost_Attribution


In [89]:
expost_factor(OptWeight['Weight'].values,upReturns[stocks],updated_ff_data[factors],betas)

,Value,Mkt-RF,SMB,HML,Mom,Alpha,Portfolio
0,TotalReturn,-0.058005,-0.001108,0.020274,0.008215,0.018044,-0.025063
1,Return Attribution,-0.044562,0.000103,0.002425,-0.000610,0.017580,-0.025063
0,Vol Attribution,0.009775,-0.000109,-0.000715,0.000127,0.002640,0.011718


## Problem3 - Risk Budget

Using the same data as Problem 1 and assuming a 0 mean return, fit a t distribution to each stock return series. Simulate the system using a Gaussian Copula. Find the Risk Parity portfolio using ES as the risk measure.

### Volatility Risk Parity Portfolio

In [90]:
def vol_risk_parity(stockMeans, covar, b=None):
    n = len(stockMeans)
    
    # Function for Portfolio Volatility
    def pvol(w):
        x = np.array(w)
        return np.sqrt(x.dot(covar).dot(x))
    
    # Function for Component Standard Deviation
    def pCSD(w, b=None, last = False):
        x = np.array(w)
        pVol = pvol(w)
        csd = x * (covar.dot(x)) / pVol
        if last:
            return csd
        if b is not None:
            csd /= b
        return csd
    
    # Sum Square Error of cSD
    def sseCSD(w):
        csd = pCSD(w, b)
        mCSD = np.sum(csd) / n
        dCsd = csd - mCSD
        se = dCsd * dCsd
        return 1.0e5 * np.sum(se) # Add a large multiplier for better convergence
    
    # Define the optimization problem
    m = minimize(sseCSD, [1/n]*n, method='SLSQP', bounds=[(0, None)]*n, constraints={'type': 'eq', 'fun': lambda w: np.sum(w)-1})
    
    w = m.x
    
    # Compute RPWeights
    RPWeights = pd.DataFrame({
        'Weight': w,
        'cEr': stockMeans * w,
        'CSD': pCSD(w, b, True)
    })
    
    return RPWeights


In [91]:
vol_risk_parity(expected_annual_return.values[0], covariance_matrix)

,Weight,cEr,CSD
AAPL,0.154203,0.021855,0.022688
MSFT,0.142566,0.024261,0.022688
BRK-B,0.265160,0.029235,0.022688
JNJ,0.287285,0.020405,0.022688
CSCO,0.150786,0.020581,0.022688


### Non Normal Risk Parity

#### Fit T-distribution with Gaussian Copula

In [92]:
from scipy.stats import t, norm

def t_distribution_copula(returns, seed=1234):
    stocks = returns.columns.tolist()

    # Fitting generalized T model for each stock
    parameters = []
    assets_returns_cdf = pd.DataFrame()

    for stock in stocks:
        params = simulation.Fitting_t_MLE(returns[stock])
        parameters.append(params)
        assets_returns_cdf[stock] = t.cdf(returns[stock],df=params[0], loc=params[1], scale = params[2])

    # Simulate N samples with spearman correlation matrix
    np.random.seed(seed)
    spearman_corr_matrix = assets_returns_cdf.corr(method='spearman')
    sim_sample = simulation.multivariate_normal_simulation(spearman_corr_matrix, 10000, method='pca')
    sim_sample = pd.DataFrame(sim_sample, columns=stocks)

    # Convert simulation result with cdf of standard normal distribution
    sim_sample_cdf = pd.DataFrame()
    for stock in stocks:
        sim_sample_cdf[stock] = norm.cdf(sim_sample[stock],loc=0,scale=1)
    
    # Convert cdf matrix to return matrix with parameters of generalized T model
    sim_returns = pd.DataFrame()
    for i, stock in enumerate(stocks):
        sim_returns[stock] = t.ppf(sim_sample_cdf[stock], df=parameters[i][0], loc=parameters[i][1], scale = parameters[i][2])
    
    return sim_returns, pd.DataFrame(parameters, columns=['df', 'loc', 'scale'], index=stocks)

In [93]:
norm_returns = all_returns[stocks] - all_returns[stocks].mean()
sim_returns, t_parameters = t_distribution_copula(norm_returns)
t_parameters

,df,loc,scale
AAPL,233.467029,0.000002,0.015827
MSFT,4.595901,0.000376,0.012400
BRK-B,21.651729,-0.000030,0.008996
JNJ,9.419772,0.000090,0.008225
CSCO,4.542758,0.000285,0.010920


In [94]:
sim_returns_1, parameters = RiskLib.simulation.gaussian_copula(norm_returns,seed=1234)

### Expected Shortfall Risk Parity

In [95]:
def es_risk_parity(stock, stockMeans, simReturn, b=None):
    # internal ES function
    def _ES(*w):

        def ES(a, alpha=0.05):
            x = np.sort(a)
            nup = int(np.ceil(a.size * alpha))
            ndn = int(np.floor(a.size * alpha))
            v = 0.5 * (x[nup] + x[ndn])
            
            es = np.mean(x[x <= v])
            return -es
        r = simReturn @ w
        return ES(r)

    # Function for the component ES
    def CES(w, b=None, last = False):
        x = list(w)
        n = len(x)
        ces = np.empty(n)
        es = _ES(*x)
        e = 1e-6
        for i in range(n):
            old = x[i]
            x[i] = x[i] + e
            ces[i] = old * (_ES(*x) - es) / e
            x[i] = old
        if last:
            return ces
        if b is not None:
            ces /= b
        return ces 

    # SSE of the Component ES
    def SSE_CES(*w):
        ces = CES(*w,b)
        ces = [x - sum(ces) / len(ces) for x in ces]
        return 1e3 * (sum([x ** 2 for x in ces]))
    
    n = len(stock)
    w0 = np.full(n, 1/n)
    bounds = [(0, None)] * n
    res = minimize(SSE_CES, w0, method='SLSQP', bounds=bounds, constraints=[{'type': 'eq', 'fun': lambda w: sum(w) - 1}])
    w = res.x
    
    # Compute RPWeights
    ES_RPWeights = pd.DataFrame({
        'Stock': stock,
        'Weight': w,
        'cEr': stockMeans * w,
        'CES': CES(w, b, True)
    }).set_index('Stock')
    
    return ES_RPWeights


In [96]:
es_risk_parity(stocks, expected_annual_return.values[0],sim_returns)

,Weight,cEr,CES
Stock,,,
AAPL,0.144883,0.020534,0.002926
MSFT,0.138463,0.023562,0.002923
BRK-B,0.276179,0.030450,0.002924
JNJ,0.299892,0.021300,0.002937
CSCO,0.140584,0.019188,0.002940


In [97]:
t_parameters

,df,loc,scale
AAPL,233.467029,0.000002,0.015827
MSFT,4.595901,0.000376,0.012400
BRK-B,21.651729,-0.000030,0.008996
JNJ,9.419772,0.000090,0.008225
CSCO,4.542758,0.000285,0.010920
